In [1]:
%%writefile inference.py
import joblib
import os
import json
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pickle
import boto3


"""
Deserialize fitted model
"""
def model_fn(model_dir):
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return input_data
    else:
        raise ValueError("This model only supports application/json input")

"""
predict_fn
    input_data: returned data from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    # Process the input data if necessary
    processed_data = process_input(input_data)
    # Make predictions using the model
    predictions = model.predict(processed_data)
    #print(predictions)
    return predictions

def process_input(input_data):
    # Process input data as needed before passing to the model for prediction
    NgramFeaturesList_pred = np.array(input_data['NgramFeaturesList_pred'])
    importsCorpus_pred = input_data['importsCorpus_pred']
    sectionNames_pred = input_data['sectionNames_pred']
    numSections_pred = int(input_data['numSections_pred'])
    

    # Load featurizers
    imports_featurizer = joblib.load(os.path.join("opt/ml/model", "imports_featurizer.pkl"))
    section_names_featurizer = joblib.load(os.path.join("opt/ml/model", "section_names_featurizer.pkl"))
    #print(NgramFeaturesList_pred, importsCorpus_pred, sectionNames_pred, numSections_pred)
    #print(imports_featurizer, section_names_featurizer)
    # Transform text features
    importsCorpus_pred_transformed = imports_featurizer.transform([importsCorpus_pred])
    sectionNames_pred_transformed = section_names_featurizer.transform([sectionNames_pred])

    # Concatenate features into a single sparse matrix
    processed_data = hstack([csr_matrix(NgramFeaturesList_pred),
                             importsCorpus_pred_transformed,
                             sectionNames_pred_transformed,
                             csr_matrix([numSections_pred]).transpose()])
    #print(processed_data)
    return processed_data

"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string
"""
def output_fn(prediction, content_type):
    res = int(prediction[0])
    respJSON = {'Output': res}
    return respJSON

"""if __name__ == '__main__':
    predict_fn({'NgramFeaturesList_pred': [[24183, 3382, 304, 17, 923, 636, 358, 275, 128, 635, 358, 613, 389, 384, 448, 12, 380, 170, 307, 122, 224, 203, 51, 338, 521, 111, 395, 215, 175, 419, 264, 397, 287, 106, 487, 236, 16, 277, 459, 594, 469, 241, 155, 163, 158, 230, 215, 443, 80, 46, 44, 216, 68, 42, 36, 48, 161, 29, 240, 145, 139, 52, 20, 75, 99, 33, 224, 161, 38, 226, 729, 139, 27, 168, 19, 68, 269, 271, 236, 33, 197, 207, 337, 1114, 126, 111, 255, 175, 47, 46, 60, 318, 129, 79, 16, 223, 162, 79, 15, 157]],
 'importsCorpus_pred': "kernel32 shlwapi ole32 shell32 user32",
 'sectionNames_pred': ".text .rdata .data .rsrc .reloc",
 'numSections_pred': "5"}, model_fn(""))
"""

Writing inference.py


In [2]:
%%writefile requirements.txt
joblib
scipy
numpy
scikit-learn

Writing requirements.txt


In [3]:
!mkdir -p opt/ml/model  
!cp model.joblib opt/ml/model/model.joblib
!cp imports_featurizer.pkl opt/ml/model/imports_featurizer.pkl
!cp section_names_featurizer.pkl opt/ml/model/section_names_featurizer.pkl

In [4]:
!tar -cvpzf model.tar.gz model.joblib inference.py imports_featurizer.pkl section_names_featurizer.pkl requirements.txt opt

model.joblib
inference.py
imports_featurizer.pkl
section_names_featurizer.pkl
requirements.txt
opt/
opt/ml/
opt/ml/model/
opt/ml/model/section_names_featurizer.pkl
opt/ml/model/model.joblib
opt/ml/model/imports_featurizer.pkl


In [6]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

In [7]:
#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)
sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::730335608842:role/LabRole"

us-east-1


In [8]:
# retrieve sklearn image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=region,
    version="1.2-1",
    py_version="py3",
    instance_type="ml.t2.medium",
)

In [9]:
#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

#Upload tar.gz to bucket
model_artifacts = f"s3://{default_bucket}/model.tar.gz"
response = s3.meta.client.upload_file('model.tar.gz', default_bucket, 'model.tar.gz')

sagemaker-us-east-1-730335608842


In [10]:
#Step 1: Model Creation
model_name = "sklearn-test" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: sklearn-test2024-04-03-18-22-01
Model Arn: arn:aws:sagemaker:us-east-1:730335608842:model/sklearn-test2024-04-03-18-22-01


In [11]:
#Step 2: EPC Creation
sklearn_epc_name = "sklearn-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,
    ProductionVariants=[
        {
            "VariantName": "sklearnvariant",
            "ModelName": model_name,
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:730335608842:endpoint-config/sklearn-epc2024-04-03-18-22-04


In [12]:
#Step 3: EP Creation
endpoint_name = "sklearn-local-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=sklearn_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:730335608842:endpoint/sklearn-local-ep2024-04-03-18-22-06


In [13]:
#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'sklearn-local-ep2024-04-03-18-22-06', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:730335608842:endpoint/sklearn-local-ep2024-04-03-18-22-06', 'EndpointConfigName': 'sklearn-epc2024-04-03-18-22-04', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:0381fc23e5c6ee7a0a944240dbaf6041688cb1e439af23e0c930da072ac9e971', 'ResolutionTime': datetime.datetime(2024, 4, 3, 18, 22, 8, 135000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': d